In [1]:
import pix2pix.pix2pix as p2p
from PIL import Image
import torchvision.transforms as transforms
import torch
from pix2pix.data.image_manipulation import *
from pix2pix.data.dataloader import *
import SRGAN.SRGAN_pretrained as SR_GAN
from ISR.models import RDN

2022-01-24 16:27:31.740846: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /media/imsanskar/My files/Projects/Minor/env/lib/python3.9/site-packages/cv2/../../lib64:
2022-01-24 16:27:31.740881: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
generator = p2p.GeneratorUNet()
device = 'cuda' if torch.cuda.is_available() else 'cpu'
generator.load_state_dict(torch.load("pix2pix/saved_models/generator.pth", map_location = device))

<All keys matched successfully>

In [ ]:
generator_srgan = SR_GAN.GeneratorSRGAN().to(device)
generator_srgan.load_state_dict(torch.load("./SRGAN/saved_models/srresnet.pth", map_location=device))
generator_srgan.eval()
rdn = RDN(weights='psnr-large')

30588928/66071288 [============>.................] - ETA: 22:01

In [14]:
image = Image.open("./pix2pix/data/train/old_images/002000.jpg")
image_width = 256
image_height = 256
transform = transforms.Compose([
    transforms.ToTensor(), # transform to tensor
    transforms.Resize((image_width, image_height)) # Resize the image to constant size
])

im = transform(image)
output = np_to_pil(generator(im.unsqueeze(0))[0].detach().cpu().numpy())
transform = transforms.Compose([
    transforms.ToTensor(),
#     transforms.Resize((256, 256), Image.BICUBIC),
])
input_image = transform(output).to(device)
output_image = np_to_pil(generator_srgan(input_image.unsqueeze(0))[0].detach().cpu().numpy())
output_image = rdn.predict(np.array(image))

In [ ]:
Image.fromarray(output_image)